<a href="https://colab.research.google.com/github/sumitshekhar41/AIAgents/blob/main/Program_17_Basic_Agent_Memory_Store_and_Semantic_Recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Explanation:**
* Demonstrates how an agent stores textual memory (interactions, thoughts)
* Uses semantic retrieval to recall relevant memory based on user queries
* Forms the foundation for agents that can remember and reason over time
* Stores every user and agent message
* Adds a timestamp and role (user or agent)
* Responds using semantic recall (retrieving similar user messages from memory)
* Allows user to type show history any time to view entire chat log

In [ ]:
# This is required for storing and retrieving agent memory
!pip install chromadb --quiet
!pip install chromadb sentence-transformers --quiet

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import uuid
from datetime import datetime

In [ ]:
# Initialize ChromaDB client and collection with embedding
client = chromadb.Client()
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

memory = client.get_or_create_collection(
    name="agent_chat_history",
    embedding_function=embedding_function
)

In [ ]:
# Define helper function to get current timestamp
def current_timestamp():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# Start interactive chat loop
print("Agent is ready. Type your message.")
print("Type 'show history' to see the full chat log.")
print("Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Agent: Goodbye.")
        break

    if user_input.lower() == "show history":
        # Retrieve all stored memory
        results = memory.get(include=["metadatas", "documents"])
        print("\n--- Chat History ---")
        combined = list(zip(results['metadatas'], results['documents']))
        # Sort by timestamp
        combined.sort(key=lambda x: x[0]['timestamp'])
        for meta, doc in combined:
            print(f"{meta['timestamp']} - {meta['role'].capitalize()}: {doc}")
        print("--------------------\n")
        continue

    # Agent queries memory to find relevant context
    results = memory.query(query_texts=[user_input], n_results=1)
    if results['documents'][0]:
        agent_response = f"Previously you said: {results['documents'][0][0]}"
    else:
        agent_response = "I haven't seen anything like that before."

    print("Agent:", agent_response)

    # Store user message in memory
    memory.add(
        documents=[user_input],
        ids=[str(uuid.uuid4())],
        metadatas=[{"role": "user", "timestamp": current_timestamp()}]
    )

    # Store agent response in memory
    memory.add(
        documents=[agent_response],
        ids=[str(uuid.uuid4())],
        metadatas=[{"role": "agent", "timestamp": current_timestamp()}]
    )